# Tutorial 2

Determining system parameters from exoplanet light curves.

**Knowns:** $P,M_*,R_*$

**From the light curve:** $\Delta f,T_{dur}$

**Assumptions:**
- Circular orbit
- Knowns assumed to have negligable error
- The star is a uniform disc
- The planet has albedo 0 and no instrinsic luminosity

**We will calculate:**
- Orbital Radius ($a$)
- Orbital speed ($v$) 
- Relative sizes of the star and planet ($R_\oplus/R_*$)
- Inclination of the Orbit ($\alpha$)

## The calculations

The change in flux is related to the relative radii as:
$$\Delta f = \frac{R_{\oplus}^2}{R_{*}^2}$$

From Kepler's 3rd law we know:
$$\frac{a^3}{P^2} = \frac{GM_*}{4\pi^2}$$

We can rearrange this to solve for the orbital radius $a$:
$$a = \left [ \frac{GM_*P^2}{4\pi^2} \right ] ^{1/3}$$

The orbital velocity $v$ can be easliey calculated as:
$$v = \frac{2\pi r}{P}$$
where
$$r = \frac{M_* a}{M_* + M_\oplus}$$
Because $M_* >> M_\oplus$ we can approximate $r = a$, and thus our calculation for $v$ becomes:
$$v = \frac{2\pi a}{P}$$

The derivation for the inclination is complex and involves much trignometry.  
I used the derivation given here: https://www.paulanthonywilson.com/exoplanets/exoplanet-detection-techniques/the-exoplanet-transit-method/ and redid all the math to convince myself I understand it.  I won't write it all out here again.  The result is that the inclination from due North (i.e. $90^{\circ}$ means looking directly edge on) is calculated as:
$$i = cos^{-1}\left ( \frac{\sqrt{(R_*+R_\oplus)^2-(a~sin(\pi T_{dur}/P))^2}}{a} \right )$$
Since we care more about how far off edge-on we are than how far from North, I will report $\alpha = 90^{\circ} - i$.

I propagated the uncertaintly in $\Delta F$ and $T_{dur}$ in the standard manner.  I think I may have a made a mistake in the propagation for the inclination calculation, because those errors seem improbably small.  If I have time I will go back and figure that out before tutorial.

## The code

Since I want to do the same calculations for several light curves I decided to code all this up in Python. Below the function definitions are my results.

In [15]:
import numpy as np
import astropy.units as u
from astropy.constants import G, M_sun, R_sun

In [139]:
def calcRplanet(deltaF, Rstar, deltaFErr = 0):
    """
    Given delta F and R star (should be astromy quantities with units) calculate R planet.
    
    Error is only assumed to come into play in the deltaF term due to the parameters of the assignment.
    """
    
    Rplanet = np.sqrt(deltaF) * Rstar
    RplanetErr = Rstar/(2*np.sqrt(deltaF))*deltaFErr
    
    return Rplanet,RplanetErr

In [68]:
def calcOrbitalRad(Mstar,P):
    """
    Given the mass of the parent star and period of the planetary orbit, calculated the orbital radius.
    
    Inputs should be astropy quanities with units.
    
    No error is assumed due to the parameters of the assignment.
    """
    return ((G*Mstar*(P**2))/(4*(np.pi**2)))**(1/3)

In [18]:
def calcOrbitalSpeed(a,P):
    """
    Given the orbital radius and period, calculated the orbital speed.
    
    Inputs should be astropy quanities with units.
    
    No error is assumed due to the parameters of the assignment.
    """
    return 2*np.pi*a/P

In [145]:
def calcOrbitalInc(Rstar, Rplanet, a, P, Tdur, RplanetErr=0*u.km, TdurErr=0*u.day):
    """
    Given star and planet radii, orbital radius and period, and transit duration, 
    calculates the orbital inclination.
    
    Inputs should be astropy quanities with units.
    
    Error is only assumed to come into play in the Rplanet and Tdur terms due to the parameters of the assignment.
    """
    
    # We'll be calculating this in sections to help with the error propagation
    X = (Rstar + Rplanet)**2
    Xerr = 2*(Rstar + Rplanet) * RplanetErr
    
    Y = (a*np.sin(np.pi*u.rad*Tdur/P))**2
    Yerr = 2*a*np.sin(np.pi*u.rad*Tdur/P)*a*np.cos(np.pi*u.rad*Tdur/P)*np.pi/P*TdurErr
    
    Z = X - Y
    Zerr = np.sqrt(Xerr**2 + Yerr**2)
    
    Q = np.sqrt(Z)/a
    Qerr = Zerr/(2*a*np.sqrt(Z))
    
    i = np.pi/2*u.rad - np.arccos(Q)
    iErr = Qerr/np.sqrt(1-Q**2)
      
    return i,iErr

In [160]:
def getAllParms(parmDict, silent=False):
    """
    Given a dictionary containing the initial set of parameters:
        P,Mstar,Rstar,deltaF,Tdur
    Calclulate the additional parameters:
        Rplanet,a,v,i
    And adds them to the original dictionary.
        
    Parameters
    ----------
    parmDict : dict
        Dictionary with keys "P", "Mstar", "Rstar", "deltaF", "Tdur".
        Dictionary elements must be astrophy quantities with units (except for deltaF which is unitless)
    silent : bool, optional
        If true, suppresses stdout output.
        
    """
    
    # Calculating the additional parameters
    parmDict["Rplanet"],parmDict["RplanetErr"] = calcRplanet(parmDict["deltaF"], parmDict["Rstar"], 
                                                             parmDict["deltaFErr"])
    parmDict["Rp2Rs"] = parmDict["Rplanet"]/parmDict["Rstar"]
    parmDict["Rp2RsErr"] = parmDict["RplanetErr"]/parmDict["Rstar"]
    parmDict["a"] = calcOrbitalRad(parmDict["Mstar"],parmDict["P"])
    parmDict["v"] = calcOrbitalSpeed(parmDict["a"],parmDict["P"])
    parmDict["i"],parmDict["iErr"] = calcOrbitalInc(parmDict["Rstar"],parmDict["Rplanet"],parmDict["a"],
                                    parmDict["P"],parmDict["Tdur"],parmDict["RplanetErr"],parmDict["TdurErr"])

    if not silent:
        # Printing the results
        print("R_planet/R_star: {:.2} +/- {:.2}".format(parmDict["Rp2Rs"],parmDict["Rp2RsErr"]))
        print("Planet radius: {:.3} +/- {:.3}".format(parmDict["Rplanet"].to(u.km),parmDict["RplanetErr"].to(u.km)))
        print("Orbital radius: {:.3}".format(parmDict["a"].to(u.AU)))
        print("Orbital velocity: {:.3}".format(parmDict["v"].to(u.km/u.second)))
        print("Orbital inclination: {:.3} +/- {:.3}".format(parmDict["i"].to(u.deg),
                                                            (parmDict["iErr"]*u.rad).to(u.deg)))
        

    

## Good Light Curves

### Object 230

In [170]:
# Known parameters
object230 = {"P":3243.57*u.day,
             "Mstar":1.45*M_sun,
             "Rstar":1.591*R_sun}

# Paremeters from chart
object230["deltaF"] = 0.031
object230["deltaFErr"] = 0.002
object230["Tdur"] = 1.725*u.day
object230["TdurErr"] = 0.1*u.day

getAllParms(object230)

R_planet/R_star: 0.18 +/- 0.0057
Planet radius: 1.95e+05 km +/- 6.28e+03 km
Orbital radius: 4.85 AU
Orbital velocity: 16.3 km / s
Orbital inclination: 0.0371 deg +/- 0.0144 deg


### Object 240

In [167]:
# Known parameters
object240 = {"P":1011.84*u.day,
             "Mstar":0.82*M_sun,
             "Rstar":0.707*R_sun}

# Paremeters from chart
object240["deltaF"] = 0.026
object240["deltaFErr"] = 0.0025
object240["Tdur"] = 0.55*u.day
object240["TdurErr"] = 0.01*u.day

getAllParms(object240)

R_planet/R_star: 0.16 +/- 0.0078
Planet radius: 7.93e+04 km +/- 3.81e+03 km
Orbital radius: 1.85 AU
Orbital velocity: 19.9 km / s
Orbital inclination: 0.0668 deg +/- 0.00296 deg


## Challenging Light Curves

### Object 280

In [163]:
# Known parameters
object280 = {"P":2322.74*u.day,
             "Mstar":1.30*M_sun,
             "Rstar":1.388*R_sun}

# Paremeters from chart
object280["deltaFErr"] = 0.0025
object280["deltaF"] = 0.0225
object280["Tdur"] = 1.425*u.day
object280["TdurErr"] = 0.06*u.day

getAllParms(object280)

R_planet/R_star: 0.15 +/- 0.0083
Planet radius: 1.45e+05 km +/- 8.04e+03 km
Orbital radius: 3.75 AU
Orbital velocity: 17.5 km / s
Orbital inclination: 0.0262 deg +/- 0.0199 deg


### Object 282

In [169]:
# Known parameters
object282 = {"P":890.29*u.day,
             "Mstar":0.75*M_sun,
             "Rstar":0.604*R_sun}

# Paremeters from chart
object282["deltaF"] = 0.025
object282["deltaFErr"] = 0.003
object282["Tdur"] = 0.35*u.day
object282["TdurErr"] = 0.01*u.day

getAllParms(object282)

R_planet/R_star: 0.16 +/- 0.0095
Planet radius: 6.64e+04 km +/- 3.99e+03 km
Orbital radius: 1.65 AU
Orbital velocity: 20.1 km / s
Orbital inclination: 0.0884 deg +/- 0.00201 deg


## Nasty Data

### Object 270

In [166]:
# Known parameters
object270 = {"P":639.50*u.day,
             "Mstar":1.81*M_sun,
             "Rstar":2.099*R_sun}

# Paremeters from chart
object270["deltaF"] = 0.0018
object270["deltaFErr"] = 0.0015
object270["Tdur"] = 0.925*u.day
object270["TdurErr"] = 0.05*u.day

getAllParms(object270)

R_planet/R_star: 0.042 +/- 0.018
Planet radius: 6.19e+04 km +/- 2.58e+04 km
Orbital radius: 1.77 AU
Orbital velocity: 30.1 km / s
Orbital inclination: 0.201 deg +/- 0.0203 deg


### Object 290

In [168]:
# Known parameters
object290 = {"P":1177.07*u.day,
             "Mstar":1.89*M_sun,
             "Rstar":2.216*R_sun}

# Paremeters from chart
object290["deltaF"] = 0.0032
object290["deltaFErr"] = 0.002
object290["Tdur"] = 1.0795*u.day
object290["TdurErr"] = 0.06*u.day

getAllParms(object290)

R_planet/R_star: 0.057 +/- 0.018
Planet radius: 8.72e+04 km +/- 2.72e+04 km
Orbital radius: 2.7 AU
Orbital velocity: 24.9 km / s
Orbital inclination: 0.162 deg +/- 0.0109 deg


## Discusion

I will go through the assumptions one by one and discuss what would happen if the assumption was not valid.

- Circular orbit
  This allows us to assume that the planet is the same distance from its parent star at all times. 
  One of the affects of such an assumption is that while the planet is in front of the star, it blocks a constant area, something that would not be true is the planet varies it's distance from the star.
  Aditionally our calculations of how far the planet is from its parent would reflet only its distance from the star during it's transit which could be wildly different than it's average distance depending on where in its orbit we observe it.


- Knowns assumed to have negligable error
  Obviously if there were non-negligable error bars on all the knowns, we would be less sure of all our conclusions.
  Depending on the size of the error bars this could lead to difficulty placing a meaningful range on any parameters.
  Esitamtes of $R_*$ affect pretty much all of the calculations, particularly the ability to calculate the planet's radius.
  Measurments of $M_*$ and $P$ affect calculation of the orbital radius, which affects analysis of the planetary system becuase of how a planets distance from its parent star affects it's temperature, suseptibility to stellar weather, etc...
  The inclination calculation depends on almost all the other system properties, so it is very quickly so affected by compounding uncertainty that it become challenging to say anything meaningful about it.
     
     
- The star is a uniform disc
  By assuming the star is a uniform disc we get to assume that the flux from the star does not vary in absence of a planetary transit.  If the star is not a uniform disc we can no longer assume that. Because the star itself is also rotating there will be some sort of cyclical variation in stellar luminosity.  This must be modeled and taken into account when searching for planetary transits (usually via detrending the light curve before looking for transits).
  Additionally there may be stochastic luminosity variation such as stellar flares, particularly on younger more active stars.  These variations can't be systematically removed, but can hinder analysis particularly if one occurs close to or during a transit.  Depending on how quiescent stellar flux is being modeled/calculated these stochasic brightenings can also increase uncertaintly of the quiescent flux, or cause overestimation.


- The planet has albedo 0 and no instrinsic luminosity
  What we get from this assumption is the ability to consider only the star's luminosity, and assume that the planet blocks it perfectly. So when the planet is fully in front of the star the luminosity is the flux of the star times (the area of star we can see minus the area blocked by the planet).  If the planent has intrinsic luminosity or positive albedo, luminosity from the planet itself must also be taken into account when making this calculation.
  The result could be underestimating the size of the planet because the flux dip is lessened due to flux from the planet itself.
  
